In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# ================================
# CONFIGURAÇÕES
# ================================

VALOR_MIN = 500
VALOR_MAX = 4000

# Caminho da pasta contendo os arquivos CSV originais
pasta_entrada = r'/caminho/para/sua/pasta_com_csvs'  # ← MODIFIQUE AQUI

# Saídas
pasta_saida = 'dados_passados_pelo_codigo'
pasta_csv = os.path.join(pasta_saida, 'csv_filtrados')
pasta_png = os.path.join(pasta_saida, 'graficos_png')

# Criar pastas de saída, se não existirem
os.makedirs(pasta_csv, exist_ok=True)
os.makedirs(pasta_png, exist_ok=True)

# Inicializar lista para guardar somas de todos os arquivos
somas_gerais = []

# Listar arquivos CSV na pasta de entrada
arquivos_csv = [f for f in os.listdir(pasta_entrada) if f.endswith('.csv')]

# ================================
# PROCESSAR CADA ARQUIVO
# ================================

for idx, nome_arquivo in enumerate(sorted(arquivos_csv), start=1):
    caminho_arquivo = os.path.join(pasta_entrada, nome_arquivo)

    try:
        df = pd.read_csv(caminho_arquivo)

        if 'Valor1' not in df.columns or 'Valor2' not in df.columns:
            print(f"⚠️ Arquivo {nome_arquivo} ignorado (colunas ausentes).")
            continue

        # Filtrar valores
        def filtrar_valores(serie):
            return serie.where((serie >= VALOR_MIN) & (serie <= VALOR_MAX), other=0)

        df['Valor1_filtrado'] = filtrar_valores(df['Valor1'])
        df['Valor2_filtrado'] = filtrar_valores(df['Valor2'])

        df_filtrado = df[['Tempo (ms)', 'Valor1_filtrado', 'Valor2_filtrado']].copy()

        # Calcular médias (ignorando zeros)
        media_valor1 = df_filtrado.loc[df_filtrado['Valor1_filtrado'] != 0, 'Valor1_filtrado'].mean()
        media_valor2 = df_filtrado.loc[df_filtrado['Valor2_filtrado'] != 0, 'Valor2_filtrado'].mean()

        # Adicionar colunas de média para plotagem
        df_filtrado['Media_Valor1'] = media_valor1
        df_filtrado['Media_Valor2'] = media_valor2

        # Calcular somas originais (antes do filtro)
        soma_valor1 = df['Valor1'].sum()
        soma_valor2 = df['Valor2'].sum()

        # Salvar CSV filtrado
        nome_saida_csv = f'filtrado_{idx}.csv'
        df_filtrado.to_csv(os.path.join(pasta_csv, nome_saida_csv), index=False)

        # ======== Gráficos ========

        # Gráfico conjunto: Valor1 e Valor2
        plt.figure(figsize=(12, 6))
        plt.plot(df_filtrado['Tempo (ms)'], df_filtrado['Valor1_filtrado'], label='Valor1 Filtrado', color='blue')
        plt.plot(df_filtrado['Tempo (ms)'], df_filtrado['Valor2_filtrado'], label='Valor2 Filtrado', color='green')
        plt.axhline(y=media_valor1, color='blue', linestyle='--', label=f'Média Valor1: {media_valor1:.2f}')
        plt.axhline(y=media_valor2, color='green', linestyle='--', label=f'Média Valor2: {media_valor2:.2f}')
        plt.title(f'Sinais Filtrados - Valor1 e Valor2 - Arquivo {idx}')
        plt.xlabel('Tempo (ms)')
        plt.ylabel('Valor')
        plt.legend()
        plt.grid(True)
        plt.ylim(0, 4000)
        plt.yticks(np.arange(0, 4001, 250))
        plt.savefig(os.path.join(pasta_png, f'grafico_completo_{idx}.png'))
        plt.close()

        # Gráfico só do Valor1
        plt.figure(figsize=(12, 6))
        plt.plot(df_filtrado['Tempo (ms)'], df_filtrado['Valor1_filtrado'], label='Valor1 Filtrado', color='blue')
        plt.axhline(y=media_valor1, color='blue', linestyle='--', label=f'Média Valor1: {media_valor1:.2f}')
        plt.title(f'Sinal Filtrado - Valor1 - Arquivo {idx}')
        plt.xlabel('Tempo (ms)')
        plt.ylabel('Valor')
        plt.legend()
        plt.grid(True)
        plt.ylim(0, 4000)
        plt.yticks(np.arange(0, 4001, 250))
        plt.savefig(os.path.join(pasta_png, f'grafico_valor1_{idx}.png'))
        plt.close()

        # Gráfico só do Valor2
        plt.figure(figsize=(12, 6))
        plt.plot(df_filtrado['Tempo (ms)'], df_filtrado['Valor2_filtrado'], label='Valor2 Filtrado', color='green')
        plt.axhline(y=media_valor2, color='green', linestyle='--', label=f'Média Valor2: {media_valor2:.2f}')
        plt.title(f'Sinal Filtrado - Valor2 - Arquivo {idx}')
        plt.xlabel('Tempo (ms)')
        plt.ylabel('Valor')
        plt.legend()
        plt.grid(True)
        plt.ylim(0, 4000)
        plt.yticks(np.arange(0, 4001, 250))
        plt.savefig(os.path.join(pasta_png, f'grafico_valor2_{idx}.png'))
        plt.close()

        # Armazenar somas
        somas_gerais.append({
            'Arquivo': nome_arquivo,
            'Soma_Valor1': soma_valor1,
            'Soma_Valor2': soma_valor2
        })

        print(f"✅ Processado: {nome_arquivo} → filtrado_{idx}.csv e gráficos")

    except Exception as e:
        print(f"❌ Erro ao processar {nome_arquivo}: {e}")

# ================================
# SALVAR CSV COM TODAS AS SOMAS
# ================================

df_somas = pd.DataFrame(somas_gerais)
df_somas.to_csv(os.path.join(pasta_saida, 'emg_soma.csv'), index=False)

print("\n🎉 Todos os arquivos foram processados com sucesso!")
print(f"📁 CSVs filtrados salvos em: {pasta_csv}")
print(f"🖼️ Gráficos PNG salvos em: {pasta_png}")
print(f"📊 Somas salvas em: {os.path.join(pasta_saida, 'emg_soma.csv')}")
